In [6]:
from covid.models.rseicha import RSEICHA
import covid
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from covid.utils import pm, pc

In [7]:
# Parameters
region_name = 'Brazil'
seed = 2000
i18n = 'pt-BR'

In [8]:
# Derived variables
region = covid.Region(region_name)
covid.set_i18n(i18n)

# Relatório de propagação do COVID-19: {{ region }}

Covid-19 é uma ameaça global, XXX.

Este relatório faz uma previsão da velocidade de alastramento da epidemia de COVID-19 e do impacto desta no sistema de saúde. Fazemos algumas recomendações de ações para conter ou diminuir a velocidade de alastramento do vírus, além de recomendações sobre o tempo ideal para realizar as intervenções.

A simulação possui algumas limitações e incertezas, principalmente porque ainda não conhecemos com muita precisão alguns parâmetros epidemiológicos e clínicos da doença. Alguns destes fatores possuem enorme impacto sobre o resultado final. Dependendo do caso, pequenas variações destes parâmetros podem afetar bastante o resultado das simulações.  

## Características básicas da região

**Nome:** {{ region }}<br>
**População:** {{ region.population_size }}<br>
**Leitos de enfermaria:** {{ region.population_size }}<br>
**Leitos de UTI:** {{ region.population_size }}<br>
**Ocupação enfermaria:** {{ region.population_size }}<br>
**Ocupação UTI:** {{ region.population_size }}<br>


## Simulação referência

In [9]:
model = RSEICHA(region=region, seed=seed)
model.run()
print(model)


SIMULATION PARAMETERS (RSEICHA)

Parameters
- R0                : 2.74
- P(is symptomatic) : 14.00%


SIMULATION RESULTS (RSEICHA)

Demography
- Total population   : 212.56mi
- Recovered          : 115.19mi (54.19%)
- Fatalities (total) : 1.67mi (0.79%)
- Infected (max)     : 16.36mi (7.70%)
- Asymptomatic (max) : 100.50mi (47.28%)
- Exposed (max)      : 116.86mi (54.98%)
        

Epidemiology
- R0   : 2.74
- IFR  : 1.43%
- CFR  : 10.21%
- HFR  : 28.09%
- HCFR : 265.01%


Healthcare parameters
- Hosp. days         : 19.62mi
- ICU days           : 11.03mi
- Peak hosp. demand  : 0.77mi (3.66‰)
    x surge capacity : 7.95
    x total          : 1.59
- Peak ICU demand    : 0.27mi (1.28‰)
    x surge capacity : 27.72
    x total          : 5.54
- Hosp. collapse day : 35.40 days (2020-04-28)
- ICU collapse day   : 33.40 days (2020-04-26)


Invariants
- Sum of compartments: 212.56mi (2.17e-14%)



In [10]:
region.population_size

212559000

In [11]:
model.health_resources()

,Name,Items/day,Total
0,Mask,25,766389350
1,Mask N95,1,30655574
2,Avental impermeável,25,766389350
3,Glove,50,1532778700
4,Faceshield,1,30655574


In [12]:
import pnad

In [13]:
df = pnad.load(2012, fields=['age', 'V0102', 'V0103'])

In [14]:
df

,age,V0102,V0103
0,48.0,11000015,2
1,48.0,11000015,2
2,23.0,11000015,2
3,21.0,11000015,2
4,54.0,11000015,3
...,...,...,...
362446,51.0,53002130,14
362447,53.0,53002130,14
362448,21.0,53002130,14
362449,19.0,53002130,14


In [15]:
from pnad.enums import State, Gender

In [16]:
State.to_category(State.DF)

'MALE'